In [2]:
!pip install pyspark
!pip install findspark

In [3]:
import pyspark
import findspark
from pyspark import SparkFiles
import json
findspark.init()
from pyspark.sql.functions import isnull, when, count, col
findspark.find()
from pyspark.ml.feature import Imputer
from pyspark.sql.functions import *
from pyspark.ml.feature import Imputer
from pyspark.sql.types import * 
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkConf, SparkContext
import warnings
%matplotlib inline
from pyspark.sql.functions import *
from pyspark.sql.functions import year, month, dayofweek
warnings.filterwarnings('ignore')
from pyspark.ml.feature import StringIndexer
from datetime import datetime
from pyspark.sql.functions import col,udf
from pyspark.sql.types import DateType
from pyspark.sql.functions import isnan, when, count, col
import pymongo

In [4]:
import findspark
findspark.init()
findspark.find()
print(pyspark.__version__)

3.3.0


In [5]:
#spark = SparkSession.builder.appName("myApp").config("spark.mongodb.input.uri","mongodb://localhost:27017/starbucks.starbucks?readPreference=primaryPreferred").config("spark.mongodb.output.uri","mongodb://localhost:27017/starbucks.starbucks").config("spark.jars.packages","org.mongodb.spark:mongo-spark-connector_2.12:3.0.0").getOrCreate()

In [6]:
#Create sparksession and mongoDB connectivity
#spark = SparkSession.builder.appName("Project")\
 #       .master("local")\
  #      .config("spark.mongodb.input.uri","mongodb://localhost:27017/projectdata.projectdata")\
  #      .config("spark.mongodb.output.uri","mongodb://localhost:27017/projectdata.projectdata")\
  #      .config("spark.jars","C:/Users/vipin/anaconda3/Lib/site-packages/pyspark/jars//" "mongo-spark-connector_2.12-3.0.1.jar,bson-4.0.5.jar,mongodb-driver-core-4.0.5.jar,mongodb-driver-sync-4.0.5.jar,scala-library-2.12.11.jar").getOrCreate()
        

In [7]:
spark = SparkSession.builder.appName("myApp").config("spark.mongodb.input.uri","mongodb://localhost:27017/starbucks.starbucks?readPreference=primaryPreferred").config("spark.mongodb.output.uri","mongodb://localhost:27017/starbucks.starbucks").config("spark.jars.packages","org.mongodb.spark:mongo-spark-connector_2.12:3.0.0").getOrCreate()

# DOWNLOADING DATASETS

In [8]:
spark.sparkContext.addFile("https://raw.githubusercontent.com/reachanihere/Starbucks-Capstone/master/data/portfolio.json")
portfolio=spark.read.json(SparkFiles.get("portfolio.json"))
portfolio=portfolio.withColumnRenamed('id', 'offer_id')
#portfolio.show()

In [9]:
#Dumping the data in mongodb
#ml = spark.read.format("mongo").option("uri","mongodb://localhost:27017/starbucks.data_for_ml").load()
#portfolio.write.format("mongo").option("uri","mongodb://localhost:27017/starbucks.data_for_ml").save()

In [10]:
spark.sparkContext.addFile("https://raw.githubusercontent.com/reachanihere/Starbucks-Capstone/master/data/transcript.json")
transcript=spark.read.json(SparkFiles.get("transcript.json"))
#transcript.show()

In [11]:
transcript=transcript.select('event','person','time','value.offer id','value.amount','value.reward')
transcript=transcript.withColumnRenamed('offer id', 'offer_id')
transcript=transcript.withColumnRenamed('person', 'customer_id')
transcript.show()

+--------------+--------------------+----+--------------------+------+------+
|         event|         customer_id|time|            offer_id|amount|reward|
+--------------+--------------------+----+--------------------+------+------+
|offer received|78afa995795e4d85b...|   0|9b98b8c7a33c4b65b...|  null|  null|
|offer received|a03223e636434f42a...|   0|0b1e1539f2cc45b7b...|  null|  null|
|offer received|e2127556f4f64592b...|   0|2906b810c7d441179...|  null|  null|
|offer received|8ec6ce2a7e7949b1b...|   0|fafdcd668e3743c1b...|  null|  null|
|offer received|68617ca6246f4fbc8...|   0|4d5c57ea9a6940dd8...|  null|  null|
|offer received|389bc3fa690240e79...|   0|f19421c1d4aa40978...|  null|  null|
|offer received|c4863c7985cf408fa...|   0|2298d6c36e964ae4a...|  null|  null|
|offer received|2eeac8d8feae4a8ca...|   0|3f207df678b143eea...|  null|  null|
|offer received|aa4862eba776480b8...|   0|0b1e1539f2cc45b7b...|  null|  null|
|offer received|31dda685af34476ca...|   0|0b1e1539f2cc45b7b...| 

In [12]:
spark.sparkContext.addFile("https://raw.githubusercontent.com/ravishchawla/DataAnalysisWithStarbucks/master/data/profile.json")
profile=spark.read.json(SparkFiles.get("profile.json"))
profile=profile.withColumnRenamed('id', 'customer_id')
#profile.show()

# Joining Table

In [13]:
transcript1=transcript.withColumnRenamed('customer_id', 'customer_id1')

joiningTable=transcript1.join(profile,
               transcript1.customer_id1 == profile.customer_id, 
               "left")
joiningTable=joiningTable.drop(col("customer_id"))
joiningTable=joiningTable.select('customer_id1','offer_id')
joiningTable=joiningTable.withColumnRenamed('offer_id', 'offer_id1')
joiningTable.show()

+--------------------+--------------------+
|        customer_id1|           offer_id1|
+--------------------+--------------------+
|78afa995795e4d85b...|9b98b8c7a33c4b65b...|
|a03223e636434f42a...|0b1e1539f2cc45b7b...|
|e2127556f4f64592b...|2906b810c7d441179...|
|8ec6ce2a7e7949b1b...|fafdcd668e3743c1b...|
|68617ca6246f4fbc8...|4d5c57ea9a6940dd8...|
|389bc3fa690240e79...|f19421c1d4aa40978...|
|c4863c7985cf408fa...|2298d6c36e964ae4a...|
|2eeac8d8feae4a8ca...|3f207df678b143eea...|
|aa4862eba776480b8...|0b1e1539f2cc45b7b...|
|31dda685af34476ca...|0b1e1539f2cc45b7b...|
|744d603ef08c4f33a...|0b1e1539f2cc45b7b...|
|3d02345581554e81b...|0b1e1539f2cc45b7b...|
|4b0da7e80e5945209...|ae264e3637204a6fb...|
|c27e0d6ab72c455a8...|3f207df678b143eea...|
|d53717f5400c4e84a...|0b1e1539f2cc45b7b...|
|f806632c011441378...|fafdcd668e3743c1b...|
|d058f73bf8674a26a...|0b1e1539f2cc45b7b...|
|65aba5c617294649a...|2906b810c7d441179...|
|ebe7ef46ea6f4963a...|9b98b8c7a33c4b65b...|
|1e9420836d554513a...|ae264e3637

# EDITING OF PROFILE

In [14]:
age118=profile.filter(col("age") == 118)
profile.filter(col("age") == 118).count()

2175

In [15]:
profile=profile.withColumnRenamed("id","customer_id")
profile.show()

+---+----------------+------+--------------------+------+
|age|became_member_on|gender|         customer_id|income|
+---+----------------+------+--------------------+------+
|118|        20170212|  null|68be06ca386d4c319...|  null|
| 55|        20170715|     F|0610b486422d4921a...|112000|
|118|        20180712|  null|38fe809add3b4fcf9...|  null|
| 75|        20170509|     F|78afa995795e4d85b...|100000|
|118|        20170804|  null|a03223e636434f42a...|  null|
| 68|        20180426|     M|e2127556f4f64592b...| 70000|
|118|        20170925|  null|8ec6ce2a7e7949b1b...|  null|
|118|        20171002|  null|68617ca6246f4fbc8...|  null|
| 65|        20180209|     M|389bc3fa690240e79...| 53000|
|118|        20161122|  null|8974fc5686fe429db...|  null|
|118|        20170824|  null|c4863c7985cf408fa...|  null|
|118|        20150919|  null|148adfcaa27d485b8...|  null|
| 58|        20171111|     M|2eeac8d8feae4a8ca...| 51000|
| 61|        20170911|     F|aa4862eba776480b8...| 57000|
| 26|        2

In [16]:
func =  udf (lambda x: datetime.strptime(x, '%Y%m%d'), DateType())

profile=profile.withColumn('became_member_on', date_format(func(col('became_member_on')), 'yyyy-MM-dd'))
#profile.show()


In [17]:
profile = profile.withColumn('dayOfWeek', dayofweek(col('became_member_on')))
profile = profile.withColumn('joining_month', month(col('became_member_on')))
profile = profile.withColumn('joining_year', year(col('became_member_on')))
#profile.show()

In [18]:
profile=profile.select(
      col("age"),col("became_member_on"),col("gender"),col("customer_id"),col("income"),col("dayOfWeek"),col("joining_month"),col("joining_year"),
      current_date().alias("current_date"),
      datediff(current_date(),col("became_member_on")).alias("datediff")
    )
profile.show()

+---+----------------+------+--------------------+------+---------+-------------+------------+------------+--------+
|age|became_member_on|gender|         customer_id|income|dayOfWeek|joining_month|joining_year|current_date|datediff|
+---+----------------+------+--------------------+------+---------+-------------+------------+------------+--------+
|118|      2017-02-12|  null|68be06ca386d4c319...|  null|        1|            2|        2017|  2022-09-21|    2047|
| 55|      2017-07-15|     F|0610b486422d4921a...|112000|        7|            7|        2017|  2022-09-21|    1894|
|118|      2018-07-12|  null|38fe809add3b4fcf9...|  null|        5|            7|        2018|  2022-09-21|    1532|
| 75|      2017-05-09|     F|78afa995795e4d85b...|100000|        3|            5|        2017|  2022-09-21|    1961|
|118|      2017-08-04|  null|a03223e636434f42a...|  null|        6|            8|        2017|  2022-09-21|    1874|
| 68|      2018-04-26|     M|e2127556f4f64592b...| 70000|       

In [19]:
profile.select([count(when(isnull(c), c)).alias(c) for c in profile.columns]).show()

+---+----------------+------+-----------+------+---------+-------------+------------+------------+--------+
|age|became_member_on|gender|customer_id|income|dayOfWeek|joining_month|joining_year|current_date|datediff|
+---+----------------+------+-----------+------+---------+-------------+------------+------------+--------+
|  0|               0|  2175|          0|  2175|        0|            0|           0|           0|       0|
+---+----------------+------+-----------+------+---------+-------------+------------+------------+--------+



In [20]:
age118=profile.filter(col("age") == 118)
#where age==118 gender and income is not given
age118.show()

+---+----------------+------+--------------------+------+---------+-------------+------------+------------+--------+
|age|became_member_on|gender|         customer_id|income|dayOfWeek|joining_month|joining_year|current_date|datediff|
+---+----------------+------+--------------------+------+---------+-------------+------------+------------+--------+
|118|      2017-02-12|  null|68be06ca386d4c319...|  null|        1|            2|        2017|  2022-09-21|    2047|
|118|      2018-07-12|  null|38fe809add3b4fcf9...|  null|        5|            7|        2018|  2022-09-21|    1532|
|118|      2017-08-04|  null|a03223e636434f42a...|  null|        6|            8|        2017|  2022-09-21|    1874|
|118|      2017-09-25|  null|8ec6ce2a7e7949b1b...|  null|        2|            9|        2017|  2022-09-21|    1822|
|118|      2017-10-02|  null|68617ca6246f4fbc8...|  null|        2|           10|        2017|  2022-09-21|    1815|
|118|      2016-11-22|  null|8974fc5686fe429db...|  null|       

In [21]:
profile.groupBy('gender').count().show()

+------+-----+
|gender|count|
+------+-----+
|     F| 6129|
|  null| 2175|
|     M| 8484|
|     O|  212|
+------+-----+



In [22]:
age118.groupBy('income').count().show()

+------+-----+
|income|count|
+------+-----+
|  null| 2175|
+------+-----+



In [23]:
profile=profile.na.drop()
#profile.show()

In [24]:
profile.count()

14825

In [25]:
'''gender_indexer = StringIndexer(inputCol="gender", outputCol="genderIndex")
profile = gender_indexer.fit(profile).transform(profile)
profile.show()'''

'gender_indexer = StringIndexer(inputCol="gender", outputCol="genderIndex")\nprofile = gender_indexer.fit(profile).transform(profile)\nprofile.show()'

In [26]:
profile=profile.withColumn('new_age', when((col('age') > lit(0)) & (col('age') < lit(10)),'age_10')
                   .when((col('age') > lit(11)) & (col('age') < lit(20)),'age_20')
                   .when((col('age') > lit(21)) & (col('age') < lit(30)),'age_30')
                   .when((col('age') > lit(31)) & (col('age') < lit(40)),'age_40')
                   .when((col('age') > lit(41)) & (col('age') < lit(50)),'age_50')
                   .when((col('age') > lit(51)) & (col('age') < lit(60)),'age_60')
                   .when((col('age') > lit(61)) & (col('age') < lit(70)),'age_70')
                   .when((col('age') > lit(71)) & (col('age') < lit(80)),'age_80')
                   .when((col('age') > lit(81)) & (col('age') < lit(90)),'age_90')
                   .when((col('age') > lit(91)) & (col('age') < lit(100)),'age_100')).withColumn('drop_col',lit(1))
#profile.show()

In [27]:
Original_profile=profile.select('age','became_member_on','gender','customer_id','income','joining_month','joining_year')
#Original_profile.show()

In [28]:
profile_Age=profile.groupBy('customer_id').pivot('new_age').sum('drop_col')
profile_Age=profile_Age.withColumnRenamed('customer_id', 'customer_id_age')
profile_Age=profile_Age.withColumnRenamed('null', 'age_10')
#profile_Age.show()

In [29]:
profile_Age.count()

14825

In [30]:
profile_Income=Original_profile.withColumn('new_income', when((col('income') > lit(0)) & (col('income') < lit(30000)),'income_30')
                   .when((col('income') > lit(30001)) & (col('income') < lit(40000)),'income_40')
                   .when((col('income') > lit(40001)) & (col('income') < lit(50000)),'income_50')
                   .when((col('income') > lit(50001)) & (col('income') < lit(60000)),'income_60')
                   .when((col('income') > lit(60001)) & (col('income') < lit(70000)),'income_70')
                   .when((col('income') > lit(70001)) & (col('income') < lit(80000)),'income_80')
                   .when((col('income') > lit(80001)) & (col('income') < lit(90000)),'income_90')
                   .when((col('income') > lit(90001)) & (col('income') < lit(100000)),'income_100')
                   .when((col('income') > lit(100001)) & (col('income') < lit(110000)),'income_110')
                    .when((col('income') > lit(110001)) & (col('income') < lit(120000)),'income_120')).withColumn('drop_col',lit(1))
#profile_Income.show()

In [31]:
profile_Income=profile_Income.groupBy('customer_id').pivot('new_income').sum('drop_col')
profile_Income=profile_Income.withColumnRenamed('customer_id', 'customer_id_inc')
profile_Income=profile_Income.withColumnRenamed('null', 'income_30')
#profile_Income.show()

In [32]:
profile_yr=Original_profile.withColumn('drop_col',lit(1))
profile_yr=profile_yr.groupBy('customer_id').pivot('joining_year').sum('drop_col')
profile_yr=profile_yr.withColumnRenamed('customer_id', 'customer_id_yr')
#profile_yr.show()

In [33]:
Original_profile.withColumn('drop_col',lit(1))
profile_month=profile.groupBy('customer_id').pivot('joining_month').sum('drop_col')
profile_month=profile_month.withColumnRenamed('customer_id', 'customer_id_month')
#profile_month.show()

In [34]:
profile.withColumn('drop_col',lit(1))
profile_Gender=profile.groupBy('customer_id').pivot('gender').sum('drop_col')
profile_Gender=profile_Gender.withColumnRenamed('customer_id', 'customer_id_gender')
profile_Gender.show()

+--------------------+----+----+----+
|  customer_id_gender|   F|   M|   O|
+--------------------+----+----+----+
|559943eeb754463e8...|   1|null|null|
|01e09d713abe4a36a...|   1|null|null|
|f5de20984950433d9...|null|   1|null|
|5465e844845a4a5c8...|null|   1|null|
|907e78791e0a4e58b...|null|   1|null|
|5abe3df001c14294a...|   1|null|null|
|e5321ed1b91147fa9...|   1|null|null|
|05ea8d8a95eb4b209...|   1|null|null|
|1699f732aae74e71b...|null|   1|null|
|9da3f2339b244229a...|   1|null|null|
|628d90cb330a4c42a...|   1|null|null|
|304183fd053441f0a...|null|   1|null|
|b3d1075ec03c4c188...|null|   1|null|
|08579355f46b49efa...|   1|null|null|
|f3acf14291c040a5b...|   1|null|null|
|74450de8dc654737b...|null|   1|null|
|515d4d9094964a188...|null|   1|null|
|c63d20f7b9f146da8...|   1|null|null|
|dd1069bbc7ef423c9...|null|   1|null|
|7d650753a27846f5a...|null|   1|null|
+--------------------+----+----+----+
only showing top 20 rows



In [35]:
'''distinct_values = list(profile.select("gender")
                       .distinct()
                       .toPandas()["gender"])

for distinct_value in distinct_values:
    function = udf(lambda item: 
                   1 if item == distinct_value else 0, 
                   IntegerType())
    new_column_name = "gender"+'_'+distinct_value
    profile = profile.withColumn(new_column_name, function(col("gender")))

profile.show()'''

'distinct_values = list(profile.select("gender")\n                       .distinct()\n                       .toPandas()["gender"])\n\nfor distinct_value in distinct_values:\n    function = udf(lambda item: \n                   1 if item == distinct_value else 0, \n                   IntegerType())\n    new_column_name = "gender"+\'_\'+distinct_value\n    profile = profile.withColumn(new_column_name, function(col("gender")))\n\nprofile.show()'

In [36]:
transcript.groupBy('event').count().show()

+---------------+------+
|          event| count|
+---------------+------+
|    transaction|138953|
| offer received| 76277|
|offer completed| 33579|
|   offer viewed| 57725|
+---------------+------+



# cleaned profile

In [37]:
profile_1=profile_Gender.join(profile_month,profile_Gender.customer_id_gender == profile_month.customer_id_month, "inner").drop('customer_id_month')
profile_1=profile_1.join(profile_Income,profile_1.customer_id_gender == profile_Income.customer_id_inc, "inner").drop('customer_id_inc')
profile_1=profile_1.join(profile_yr,profile_1.customer_id_gender == profile_yr.customer_id_yr, "inner").drop('customer_id_yr')
profile_1=profile_1.join(profile_Age,profile_1.customer_id_gender == profile_Age.customer_id_age, "inner").drop('customer_id_age')
profile_1=profile_1.join(Original_profile,profile_1.customer_id_gender == Original_profile.customer_id, "inner").drop('customer_id_age')\
.drop('gender','joining_month','joining_year','income','customer_id')
profile_1=profile_1.na.fill(value=0)
profile_1.show()
#profile_1.count()

+--------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---------+----------+----------+----------+---------+---------+---------+---------+---------+---------+----+----+----+----+----+----+------+-------+------+------+------+------+------+------+------+------+---+----------------+
|  customer_id_gender|  F|  M|  O|  1|  2|  3|  4|  5|  6|  7|  8|  9| 10| 11| 12|income_30|income_100|income_110|income_120|income_40|income_50|income_60|income_70|income_80|income_90|2013|2014|2015|2016|2017|2018|age_10|age_100|age_20|age_30|age_40|age_50|age_60|age_70|age_80|age_90|age|became_member_on|
+--------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---------+----------+----------+----------+---------+---------+---------+---------+---------+---------+----+----+----+----+----+----+------+-------+------+------+------+------+------+------+------+------+---+----------------+
|b3d1075ec03c4c188...|  0|  1|  0|  0|  0|  0|  0|  0|  1|  0|  0|  0|  0|  

In [38]:
profile_1.count()

14825

# PORTFOLIO DATASET

In [39]:
portfolio.show()

+--------------------+----------+--------+--------------------+-------------+------+
|            channels|difficulty|duration|            offer_id|   offer_type|reward|
+--------------------+----------+--------+--------------------+-------------+------+
|[email, mobile, s...|        10|     7.0|ae264e3637204a6fb...|         bogo|    10|
|[web, email, mobi...|        10|     5.0|4d5c57ea9a6940dd8...|         bogo|    10|
|[web, email, mobile]|         0|     4.0|3f207df678b143eea...|informational|     0|
|[web, email, mobile]|         5|     7.0|9b98b8c7a33c4b65b...|         bogo|     5|
|        [web, email]|        20|    10.0|0b1e1539f2cc45b7b...|     discount|     5|
|[web, email, mobi...|         7|     7.0|2298d6c36e964ae4a...|     discount|     3|
|[web, email, mobi...|        10|    10.0|fafdcd668e3743c1b...|     discount|     2|
|[email, mobile, s...|         0|     3.0|5a8bc65990b245e5a...|informational|     0|
|[web, email, mobi...|         5|     5.0|f19421c1d4aa40978...|  

In [40]:
portfolio11=portfolio.select('offer_id','offer_type')
portfolio11=portfolio11.withColumnRenamed('offer_id', 'offer_id1')
portfolio11.show()

+--------------------+-------------+
|           offer_id1|   offer_type|
+--------------------+-------------+
|ae264e3637204a6fb...|         bogo|
|4d5c57ea9a6940dd8...|         bogo|
|3f207df678b143eea...|informational|
|9b98b8c7a33c4b65b...|         bogo|
|0b1e1539f2cc45b7b...|     discount|
|2298d6c36e964ae4a...|     discount|
|fafdcd668e3743c1b...|     discount|
|5a8bc65990b245e5a...|informational|
|f19421c1d4aa40978...|         bogo|
|2906b810c7d441179...|     discount|
+--------------------+-------------+



In [41]:
portfolio.printSchema()

root
 |-- channels: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- difficulty: long (nullable = true)
 |-- duration: double (nullable = true)
 |-- offer_id: string (nullable = true)
 |-- offer_type: string (nullable = true)
 |-- reward: long (nullable = true)



In [42]:
prrtfolio=portfolio.na.drop()
prrtfolio.show()

+--------------------+----------+--------+--------------------+-------------+------+
|            channels|difficulty|duration|            offer_id|   offer_type|reward|
+--------------------+----------+--------+--------------------+-------------+------+
|[email, mobile, s...|        10|     7.0|ae264e3637204a6fb...|         bogo|    10|
|[web, email, mobi...|        10|     5.0|4d5c57ea9a6940dd8...|         bogo|    10|
|[web, email, mobile]|         0|     4.0|3f207df678b143eea...|informational|     0|
|[web, email, mobile]|         5|     7.0|9b98b8c7a33c4b65b...|         bogo|     5|
|        [web, email]|        20|    10.0|0b1e1539f2cc45b7b...|     discount|     5|
|[web, email, mobi...|         7|     7.0|2298d6c36e964ae4a...|     discount|     3|
|[web, email, mobi...|        10|    10.0|fafdcd668e3743c1b...|     discount|     2|
|[email, mobile, s...|         0|     3.0|5a8bc65990b245e5a...|informational|     0|
|[web, email, mobi...|         5|     5.0|f19421c1d4aa40978...|  

In [43]:
prrtfolio=prrtfolio.select(sort_array("channels",asc=True),"offer_id","duration","difficulty","offer_type","reward").withColumn('duration',col('duration')*24)
prrtfolio=prrtfolio.withColumnRenamed("sort_array(channels, true)","channels")
prrtfolio.show()

+--------------------+--------------------+--------+----------+-------------+------+
|            channels|            offer_id|duration|difficulty|   offer_type|reward|
+--------------------+--------------------+--------+----------+-------------+------+
|[email, mobile, s...|ae264e3637204a6fb...|   168.0|        10|         bogo|    10|
|[email, mobile, s...|4d5c57ea9a6940dd8...|   120.0|        10|         bogo|    10|
|[email, mobile, web]|3f207df678b143eea...|    96.0|         0|informational|     0|
|[email, mobile, web]|9b98b8c7a33c4b65b...|   168.0|         5|         bogo|     5|
|        [email, web]|0b1e1539f2cc45b7b...|   240.0|        20|     discount|     5|
|[email, mobile, s...|2298d6c36e964ae4a...|   168.0|         7|     discount|     3|
|[email, mobile, s...|fafdcd668e3743c1b...|   240.0|        10|     discount|     2|
|[email, mobile, s...|5a8bc65990b245e5a...|    72.0|         0|informational|     0|
|[email, mobile, s...|f19421c1d4aa40978...|   120.0|         5|  

In [44]:
portfolio_org=prrtfolio.drop('channels')
portfolio_org.show()

+--------------------+--------+----------+-------------+------+
|            offer_id|duration|difficulty|   offer_type|reward|
+--------------------+--------+----------+-------------+------+
|ae264e3637204a6fb...|   168.0|        10|         bogo|    10|
|4d5c57ea9a6940dd8...|   120.0|        10|         bogo|    10|
|3f207df678b143eea...|    96.0|         0|informational|     0|
|9b98b8c7a33c4b65b...|   168.0|         5|         bogo|     5|
|0b1e1539f2cc45b7b...|   240.0|        20|     discount|     5|
|2298d6c36e964ae4a...|   168.0|         7|     discount|     3|
|fafdcd668e3743c1b...|   240.0|        10|     discount|     2|
|5a8bc65990b245e5a...|    72.0|         0|informational|     0|
|f19421c1d4aa40978...|   120.0|         5|         bogo|     5|
|2906b810c7d441179...|   168.0|        10|     discount|     2|
+--------------------+--------+----------+-------------+------+



In [45]:
prrtfolio=prrtfolio.withColumn('channels',explode(col('channels'))).withColumn('drop_col',lit(1))
prrtfolio.show()

+--------+--------------------+--------+----------+-------------+------+--------+
|channels|            offer_id|duration|difficulty|   offer_type|reward|drop_col|
+--------+--------------------+--------+----------+-------------+------+--------+
|   email|ae264e3637204a6fb...|   168.0|        10|         bogo|    10|       1|
|  mobile|ae264e3637204a6fb...|   168.0|        10|         bogo|    10|       1|
|  social|ae264e3637204a6fb...|   168.0|        10|         bogo|    10|       1|
|   email|4d5c57ea9a6940dd8...|   120.0|        10|         bogo|    10|       1|
|  mobile|4d5c57ea9a6940dd8...|   120.0|        10|         bogo|    10|       1|
|  social|4d5c57ea9a6940dd8...|   120.0|        10|         bogo|    10|       1|
|     web|4d5c57ea9a6940dd8...|   120.0|        10|         bogo|    10|       1|
|   email|3f207df678b143eea...|    96.0|         0|informational|     0|       1|
|  mobile|3f207df678b143eea...|    96.0|         0|informational|     0|       1|
|     web|3f207d

In [46]:
prrtfolio.count()

33

In [47]:
prrtfolio_Channel=prrtfolio.groupBy('offer_id').pivot('channels').sum('drop_col')
prrtfolio_Channel=prrtfolio_Channel.withColumnRenamed('offer_id', 'offer_id1')
prrtfolio_Channel.show()

+--------------------+-----+------+------+----+
|           offer_id1|email|mobile|social| web|
+--------------------+-----+------+------+----+
|0b1e1539f2cc45b7b...|    1|  null|  null|   1|
|4d5c57ea9a6940dd8...|    1|     1|     1|   1|
|9b98b8c7a33c4b65b...|    1|     1|  null|   1|
|f19421c1d4aa40978...|    1|     1|     1|   1|
|fafdcd668e3743c1b...|    1|     1|     1|   1|
|ae264e3637204a6fb...|    1|     1|     1|null|
|5a8bc65990b245e5a...|    1|     1|     1|null|
|2298d6c36e964ae4a...|    1|     1|     1|   1|
|2906b810c7d441179...|    1|     1|  null|   1|
|3f207df678b143eea...|    1|     1|  null|   1|
+--------------------+-----+------+------+----+



In [48]:
prrtfolio_Channel.count()

10

In [49]:
#prrtfolio.withColumn('drop_col',lit(1))
prrtfolio_offer_type=prrtfolio.groupBy('offer_id').pivot('offer_type').sum('drop_col')
#prrtfolio_offer_type=prrtfolio_offer_type.withColumnRenamed("offer_id","offer_id1")
prrtfolio_offer_type.show()

+--------------------+----+--------+-------------+
|            offer_id|bogo|discount|informational|
+--------------------+----+--------+-------------+
|0b1e1539f2cc45b7b...|null|       2|         null|
|4d5c57ea9a6940dd8...|   4|    null|         null|
|9b98b8c7a33c4b65b...|   3|    null|         null|
|f19421c1d4aa40978...|   4|    null|         null|
|fafdcd668e3743c1b...|null|       4|         null|
|ae264e3637204a6fb...|   3|    null|         null|
|5a8bc65990b245e5a...|null|    null|            3|
|2298d6c36e964ae4a...|null|       4|         null|
|2906b810c7d441179...|null|       3|         null|
|3f207df678b143eea...|null|    null|            3|
+--------------------+----+--------+-------------+



In [50]:
prrtfolio_offer_type.na.fill(value=0).show()

+--------------------+----+--------+-------------+
|            offer_id|bogo|discount|informational|
+--------------------+----+--------+-------------+
|0b1e1539f2cc45b7b...|   0|       2|            0|
|4d5c57ea9a6940dd8...|   4|       0|            0|
|9b98b8c7a33c4b65b...|   3|       0|            0|
|f19421c1d4aa40978...|   4|       0|            0|
|fafdcd668e3743c1b...|   0|       4|            0|
|ae264e3637204a6fb...|   3|       0|            0|
|5a8bc65990b245e5a...|   0|       0|            3|
|2298d6c36e964ae4a...|   0|       4|            0|
|2906b810c7d441179...|   0|       3|            0|
|3f207df678b143eea...|   0|       0|            3|
+--------------------+----+--------+-------------+



In [51]:
prrtfolio_offer_type = prrtfolio_offer_type.withColumn("bogo", when(prrtfolio_offer_type.bogo > 0,lit(1)) \
      .otherwise(prrtfolio_offer_type.bogo))\
    .withColumn("discount", when(prrtfolio_offer_type.discount > 0,lit(1)) \
      .otherwise(prrtfolio_offer_type.discount))\
.withColumn("informational", when(prrtfolio_offer_type.informational > 0,lit(1)) \
      .otherwise(prrtfolio_offer_type.informational))
prrtfolio_offer_type.show()

+--------------------+----+--------+-------------+
|            offer_id|bogo|discount|informational|
+--------------------+----+--------+-------------+
|0b1e1539f2cc45b7b...|null|       1|         null|
|4d5c57ea9a6940dd8...|   1|    null|         null|
|9b98b8c7a33c4b65b...|   1|    null|         null|
|f19421c1d4aa40978...|   1|    null|         null|
|fafdcd668e3743c1b...|null|       1|         null|
|ae264e3637204a6fb...|   1|    null|         null|
|5a8bc65990b245e5a...|null|    null|            1|
|2298d6c36e964ae4a...|null|       1|         null|
|2906b810c7d441179...|null|       1|         null|
|3f207df678b143eea...|null|    null|            1|
+--------------------+----+--------+-------------+



In [52]:
prrtfolio_offer_type.count()

10

In [53]:
prrtfolioANDjoiningTable=joiningTable.join(prrtfolio_offer_type,
               joiningTable.offer_id1 == prrtfolio_offer_type.offer_id, 
               "left")
prrtfolioANDjoiningTable.show()

+--------------------+--------------------+--------------------+----+--------+-------------+
|        customer_id1|           offer_id1|            offer_id|bogo|discount|informational|
+--------------------+--------------------+--------------------+----+--------+-------------+
|78afa995795e4d85b...|9b98b8c7a33c4b65b...|9b98b8c7a33c4b65b...|   1|    null|         null|
|a03223e636434f42a...|0b1e1539f2cc45b7b...|0b1e1539f2cc45b7b...|null|       1|         null|
|e2127556f4f64592b...|2906b810c7d441179...|2906b810c7d441179...|null|       1|         null|
|8ec6ce2a7e7949b1b...|fafdcd668e3743c1b...|fafdcd668e3743c1b...|null|       1|         null|
|68617ca6246f4fbc8...|4d5c57ea9a6940dd8...|4d5c57ea9a6940dd8...|   1|    null|         null|
|389bc3fa690240e79...|f19421c1d4aa40978...|f19421c1d4aa40978...|   1|    null|         null|
|c4863c7985cf408fa...|2298d6c36e964ae4a...|2298d6c36e964ae4a...|null|       1|         null|
|2eeac8d8feae4a8ca...|3f207df678b143eea...|3f207df678b143eea...|null| 

In [54]:
prrtfolioANDjoiningTable.count()

306534

In [55]:
prrtfolioANDjoiningTable=prrtfolioANDjoiningTable.drop(col('customer_id1'))
prrtfolioANDjoiningTable=prrtfolioANDjoiningTable.drop(col('offer_id1'))
prrtfolioANDjoiningTable.show()

+--------------------+----+--------+-------------+
|            offer_id|bogo|discount|informational|
+--------------------+----+--------+-------------+
|9b98b8c7a33c4b65b...|   1|    null|         null|
|0b1e1539f2cc45b7b...|null|       1|         null|
|2906b810c7d441179...|null|       1|         null|
|fafdcd668e3743c1b...|null|       1|         null|
|4d5c57ea9a6940dd8...|   1|    null|         null|
|f19421c1d4aa40978...|   1|    null|         null|
|2298d6c36e964ae4a...|null|       1|         null|
|3f207df678b143eea...|null|    null|            1|
|0b1e1539f2cc45b7b...|null|       1|         null|
|0b1e1539f2cc45b7b...|null|       1|         null|
|0b1e1539f2cc45b7b...|null|       1|         null|
|0b1e1539f2cc45b7b...|null|       1|         null|
|ae264e3637204a6fb...|   1|    null|         null|
|3f207df678b143eea...|null|    null|            1|
|0b1e1539f2cc45b7b...|null|       1|         null|
|fafdcd668e3743c1b...|null|       1|         null|
|0b1e1539f2cc45b7b...|null|    

In [56]:
'''combinedDF22=prrtfolio_offer_type.join(prrtfolio_Channel,
               prrtfolio_offer_type.offer_id1 == prrtfolio_Channel.offer_id, 
               "left")
combinedDF22=combinedDF22.drop(col("offer_id"))
combinedDF22.show()'''

'combinedDF22=prrtfolio_offer_type.join(prrtfolio_Channel,\n               prrtfolio_offer_type.offer_id1 == prrtfolio_Channel.offer_id, \n               "left")\ncombinedDF22=combinedDF22.drop(col("offer_id"))\ncombinedDF22.show()'

In [57]:
'''combinedDF222=combinedDF22.join(transcript,
               combinedDF22.offer_id1 == transcript.offer_id, 
               "left")
combinedDF222.show()'''

'combinedDF222=combinedDF22.join(transcript,\n               combinedDF22.offer_id1 == transcript.offer_id, \n               "left")\ncombinedDF222.show()'

In [58]:
prrtfolio.show()

+--------+--------------------+--------+----------+-------------+------+--------+
|channels|            offer_id|duration|difficulty|   offer_type|reward|drop_col|
+--------+--------------------+--------+----------+-------------+------+--------+
|   email|ae264e3637204a6fb...|   168.0|        10|         bogo|    10|       1|
|  mobile|ae264e3637204a6fb...|   168.0|        10|         bogo|    10|       1|
|  social|ae264e3637204a6fb...|   168.0|        10|         bogo|    10|       1|
|   email|4d5c57ea9a6940dd8...|   120.0|        10|         bogo|    10|       1|
|  mobile|4d5c57ea9a6940dd8...|   120.0|        10|         bogo|    10|       1|
|  social|4d5c57ea9a6940dd8...|   120.0|        10|         bogo|    10|       1|
|     web|4d5c57ea9a6940dd8...|   120.0|        10|         bogo|    10|       1|
|   email|3f207df678b143eea...|    96.0|         0|informational|     0|       1|
|  mobile|3f207df678b143eea...|    96.0|         0|informational|     0|       1|
|     web|3f207d

In [59]:
Original_portfolio=prrtfolio.withColumnRenamed("offer_id","offer_id1")
Original_portfolio.show()

+--------+--------------------+--------+----------+-------------+------+--------+
|channels|           offer_id1|duration|difficulty|   offer_type|reward|drop_col|
+--------+--------------------+--------+----------+-------------+------+--------+
|   email|ae264e3637204a6fb...|   168.0|        10|         bogo|    10|       1|
|  mobile|ae264e3637204a6fb...|   168.0|        10|         bogo|    10|       1|
|  social|ae264e3637204a6fb...|   168.0|        10|         bogo|    10|       1|
|   email|4d5c57ea9a6940dd8...|   120.0|        10|         bogo|    10|       1|
|  mobile|4d5c57ea9a6940dd8...|   120.0|        10|         bogo|    10|       1|
|  social|4d5c57ea9a6940dd8...|   120.0|        10|         bogo|    10|       1|
|     web|4d5c57ea9a6940dd8...|   120.0|        10|         bogo|    10|       1|
|   email|3f207df678b143eea...|    96.0|         0|informational|     0|       1|
|  mobile|3f207df678b143eea...|    96.0|         0|informational|     0|       1|
|     web|3f207d

# cleaned portfolio

In [60]:
portfolio_1=prrtfolio_offer_type.join(prrtfolio_Channel,prrtfolio_offer_type.offer_id == prrtfolio_Channel.offer_id1, "inner").drop('offer_id1')
portfolio_1=portfolio_1.join(portfolio_org,["offer_id"], "inner").drop('offer_type','channels','reward')
portfolio_1=portfolio_1.na.fill(value=0)
portfolio_1.show()

+--------------------+----+--------+-------------+-----+------+------+---+--------+----------+
|            offer_id|bogo|discount|informational|email|mobile|social|web|duration|difficulty|
+--------------------+----+--------+-------------+-----+------+------+---+--------+----------+
|0b1e1539f2cc45b7b...|   0|       1|            0|    1|     0|     0|  1|   240.0|        20|
|4d5c57ea9a6940dd8...|   1|       0|            0|    1|     1|     1|  1|   120.0|        10|
|9b98b8c7a33c4b65b...|   1|       0|            0|    1|     1|     0|  1|   168.0|         5|
|f19421c1d4aa40978...|   1|       0|            0|    1|     1|     1|  1|   120.0|         5|
|fafdcd668e3743c1b...|   0|       1|            0|    1|     1|     1|  1|   240.0|        10|
|ae264e3637204a6fb...|   1|       0|            0|    1|     1|     1|  0|   168.0|        10|
|5a8bc65990b245e5a...|   0|       0|            1|    1|     1|     1|  0|    72.0|         0|
|2298d6c36e964ae4a...|   0|       1|            0|

In [61]:
portfolio_1.count()

10

# TRANCSRIPT DATASET

In [62]:
transcript.show()

+--------------+--------------------+----+--------------------+------+------+
|         event|         customer_id|time|            offer_id|amount|reward|
+--------------+--------------------+----+--------------------+------+------+
|offer received|78afa995795e4d85b...|   0|9b98b8c7a33c4b65b...|  null|  null|
|offer received|a03223e636434f42a...|   0|0b1e1539f2cc45b7b...|  null|  null|
|offer received|e2127556f4f64592b...|   0|2906b810c7d441179...|  null|  null|
|offer received|8ec6ce2a7e7949b1b...|   0|fafdcd668e3743c1b...|  null|  null|
|offer received|68617ca6246f4fbc8...|   0|4d5c57ea9a6940dd8...|  null|  null|
|offer received|389bc3fa690240e79...|   0|f19421c1d4aa40978...|  null|  null|
|offer received|c4863c7985cf408fa...|   0|2298d6c36e964ae4a...|  null|  null|
|offer received|2eeac8d8feae4a8ca...|   0|3f207df678b143eea...|  null|  null|
|offer received|aa4862eba776480b8...|   0|0b1e1539f2cc45b7b...|  null|  null|
|offer received|31dda685af34476ca...|   0|0b1e1539f2cc45b7b...| 

In [63]:
transcript.count()

306534

In [64]:
transcript.printSchema()

root
 |-- event: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- time: long (nullable = true)
 |-- offer_id: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- reward: long (nullable = true)



In [65]:
#actual_transcript=transcript.withColumnRenamed("customer_id","customer_id1")
actual_transcript=transcript.withColumnRenamed("offer id","offer_id")
actual_transcript=actual_transcript.na.fill(value=0)
actual_transcript.show()

+--------------+--------------------+----+--------------------+------+------+
|         event|         customer_id|time|            offer_id|amount|reward|
+--------------+--------------------+----+--------------------+------+------+
|offer received|78afa995795e4d85b...|   0|9b98b8c7a33c4b65b...|   0.0|     0|
|offer received|a03223e636434f42a...|   0|0b1e1539f2cc45b7b...|   0.0|     0|
|offer received|e2127556f4f64592b...|   0|2906b810c7d441179...|   0.0|     0|
|offer received|8ec6ce2a7e7949b1b...|   0|fafdcd668e3743c1b...|   0.0|     0|
|offer received|68617ca6246f4fbc8...|   0|4d5c57ea9a6940dd8...|   0.0|     0|
|offer received|389bc3fa690240e79...|   0|f19421c1d4aa40978...|   0.0|     0|
|offer received|c4863c7985cf408fa...|   0|2298d6c36e964ae4a...|   0.0|     0|
|offer received|2eeac8d8feae4a8ca...|   0|3f207df678b143eea...|   0.0|     0|
|offer received|aa4862eba776480b8...|   0|0b1e1539f2cc45b7b...|   0.0|     0|
|offer received|31dda685af34476ca...|   0|0b1e1539f2cc45b7b...| 

In [66]:
a=transcript.groupBy('customer_id').count()
a.count()

17000

In [67]:
transcript.count()

306534

In [68]:
transcript.groupBy('event').count().show()

+---------------+------+
|          event| count|
+---------------+------+
|    transaction|138953|
| offer received| 76277|
|offer completed| 33579|
|   offer viewed| 57725|
+---------------+------+



In [69]:
transcript.select([count(when(isnan(c), c)).alias(c) for c in transcript.columns]).show()

+-----+-----------+----+--------+------+------+
|event|customer_id|time|offer_id|amount|reward|
+-----+-----------+----+--------+------+------+
|    0|          0|   0|       0|     0|     0|
+-----+-----------+----+--------+------+------+



In [70]:
transcript=transcript.drop(col("reward"))

In [71]:
transcript.groupBy('event').count().show()

+---------------+------+
|          event| count|
+---------------+------+
|    transaction|138953|
| offer received| 76277|
|offer completed| 33579|
|   offer viewed| 57725|
+---------------+------+



In [72]:
transcript_0=transcript.withColumnRenamed("customer_id","customer_id1").withColumn('drop_col',lit(1))
transcript_event=transcript_0.groupBy('customer_id1').pivot('event').sum('drop_col')
transcript_event=transcript_event.na.fill(value=0)
#transcript_event=transcript_event.na.drop()
transcript_event.show()

+--------------------+---------------+--------------+------------+-----------+
|        customer_id1|offer completed|offer received|offer viewed|transaction|
+--------------------+---------------+--------------+------------+-----------+
|5e0cac0673884c67b...|              0|             4|           2|          8|
|d3c24fa42d0947a4b...|              3|             4|           2|         15|
|b3d1075ec03c4c188...|              4|             4|           4|          5|
|5abe3df001c14294a...|              4|             5|           5|         10|
|f6c178ca2b1847d6b...|              5|             5|           5|          6|
|f608dc84f5f744a88...|              3|             5|           3|          4|
|e1e4b47968ef42509...|              2|             4|           3|          5|
|3625554b70d8411bb...|              3|             4|           4|          9|
|7b68d6af297344258...|              2|             3|           3|         10|
|32a0f871e17c4cfdb...|              3|             4

In [73]:
transcript_event.count()

17000

In [74]:
transcript_event=transcript_event.withColumnRenamed("offer completed","offer_completed")\
.withColumnRenamed("offer received","offer_received")\
.withColumnRenamed("offer viewed","offer_viewed")

In [75]:
transcript_event = transcript_event.withColumn("offer_completed", when(transcript_event.offer_completed > 0,lit(1)) \
      .otherwise(transcript_event.offer_completed))\
         .withColumn("offer_received", when(transcript_event.offer_received > 0,lit(1)) \
      .otherwise(transcript_event.offer_received))\
.withColumn("offer_viewed", when(transcript_event.offer_viewed > 0,lit(1)) \
      .otherwise(transcript_event.offer_viewed))\
.withColumn("transaction", when(transcript_event.transaction > 0,lit(1)) \
      .otherwise(transcript_event.transaction))
#transcript_event=transcript_event.withColumnRenamed("offer_id","offer_id1") 
transcript_event.show()

+--------------------+---------------+--------------+------------+-----------+
|        customer_id1|offer_completed|offer_received|offer_viewed|transaction|
+--------------------+---------------+--------------+------------+-----------+
|5e0cac0673884c67b...|              0|             1|           1|          1|
|d3c24fa42d0947a4b...|              1|             1|           1|          1|
|b3d1075ec03c4c188...|              1|             1|           1|          1|
|5abe3df001c14294a...|              1|             1|           1|          1|
|f6c178ca2b1847d6b...|              1|             1|           1|          1|
|f608dc84f5f744a88...|              1|             1|           1|          1|
|e1e4b47968ef42509...|              1|             1|           1|          1|
|3625554b70d8411bb...|              1|             1|           1|          1|
|7b68d6af297344258...|              1|             1|           1|          1|
|32a0f871e17c4cfdb...|              1|             1

In [76]:
transcript_event.count()

17000

# cleaned transcript

In [77]:
transcript_1=transcript_event.join(actual_transcript,transcript_event.customer_id1 == actual_transcript.customer_id, "inner")\
.drop('customer_id1')
transcript_1=transcript_1.na.fill(value=0)
transcript_1.show()

+---------------+--------------+------------+-----------+--------------+--------------------+----+--------------------+------+------+
|offer_completed|offer_received|offer_viewed|transaction|         event|         customer_id|time|            offer_id|amount|reward|
+---------------+--------------+------------+-----------+--------------+--------------------+----+--------------------+------+------+
|              1|             1|           1|          1|offer received|c05652fa9d2c4fac9...|   0|9b98b8c7a33c4b65b...|   0.0|     0|
|              1|             1|           1|          1|offer received|9da3f2339b244229a...|   0|4d5c57ea9a6940dd8...|   0.0|     0|
|              0|             1|           1|          1|offer received|fc58860830e648c9b...|   0|9b98b8c7a33c4b65b...|   0.0|     0|
|              1|             1|           1|          1|offer received|f608dc84f5f744a88...|   0|9b98b8c7a33c4b65b...|   0.0|     0|
|              1|             1|           1|          1|offer

In [78]:
transcript_1.count()

306534

# COMBINED DATASET

In [79]:
actual_transcript=actual_transcript.withColumnRenamed("customer_id","customer_id_tr")  

In [80]:
#actual_transcript.count()
#Original_profile.count()
Original_portfolio.count()

33

In [81]:
combined_data=actual_transcript.join(Original_profile,Original_profile.customer_id == actual_transcript.customer_id_tr, "inner")\
.drop('customer_id_tr')
combined_data=combined_data.join(Original_portfolio,combined_data.offer_id == Original_portfolio.offer_id1, "inner")\
.drop('offer_id1','index_column')
combined_data=combined_data.na.fill(value=0)
combined_data.show()

+--------------+----+--------------------+------+------+---+----------------+------+--------------------+------+-------------+------------+--------+--------+----------+-------------+------+--------+
|         event|time|            offer_id|amount|reward|age|became_member_on|gender|         customer_id|income|joining_month|joining_year|channels|duration|difficulty|   offer_type|reward|drop_col|
+--------------+----+--------------------+------+------+---+----------------+------+--------------------+------+-------------+------------+--------+--------+----------+-------------+------+--------+
|offer received|   0|9b98b8c7a33c4b65b...|   0.0|     0| 75|      2017-05-09|     F|78afa995795e4d85b...|100000|            5|        2017|     web|   168.0|         5|         bogo|     5|       1|
|offer received|   0|9b98b8c7a33c4b65b...|   0.0|     0| 75|      2017-05-09|     F|78afa995795e4d85b...|100000|            5|        2017|  mobile|   168.0|         5|         bogo|     5|       1|
|offe

In [82]:
combined_data.count()

392043

# PROCESSING

In [83]:
prrtfolio_offer_type=prrtfolio_offer_type.withColumnRenamed('offer_id', 'offer_id1')
prrtfolio_offer_type.show()

+--------------------+----+--------+-------------+
|           offer_id1|bogo|discount|informational|
+--------------------+----+--------+-------------+
|0b1e1539f2cc45b7b...|null|       1|         null|
|4d5c57ea9a6940dd8...|   1|    null|         null|
|9b98b8c7a33c4b65b...|   1|    null|         null|
|f19421c1d4aa40978...|   1|    null|         null|
|fafdcd668e3743c1b...|null|       1|         null|
|ae264e3637204a6fb...|   1|    null|         null|
|5a8bc65990b245e5a...|null|    null|            1|
|2298d6c36e964ae4a...|null|       1|         null|
|2906b810c7d441179...|null|       1|         null|
|3f207df678b143eea...|null|    null|            1|
+--------------------+----+--------+-------------+



In [84]:
transcript_event=transcript_event.withColumnRenamed('offer_id', 'offer_id1')
transcript_event.show()

+--------------------+---------------+--------------+------------+-----------+
|        customer_id1|offer_completed|offer_received|offer_viewed|transaction|
+--------------------+---------------+--------------+------------+-----------+
|5e0cac0673884c67b...|              0|             1|           1|          1|
|d3c24fa42d0947a4b...|              1|             1|           1|          1|
|b3d1075ec03c4c188...|              1|             1|           1|          1|
|5abe3df001c14294a...|              1|             1|           1|          1|
|f6c178ca2b1847d6b...|              1|             1|           1|          1|
|f608dc84f5f744a88...|              1|             1|           1|          1|
|e1e4b47968ef42509...|              1|             1|           1|          1|
|3625554b70d8411bb...|              1|             1|           1|          1|
|7b68d6af297344258...|              1|             1|           1|          1|
|32a0f871e17c4cfdb...|              1|             1

In [85]:
transcript_event.count()

17000

In [86]:
tr_event=transcript_event.join(transcript,
               transcript_event.customer_id1 == transcript.customer_id, 
               "inner")
tr_event=tr_event.select('customer_id1','offer_id','offer_completed','offer_received','offer_viewed','transaction','amount')
tr_event.show()

+--------------------+--------------------+---------------+--------------+------------+-----------+------+
|        customer_id1|            offer_id|offer_completed|offer_received|offer_viewed|transaction|amount|
+--------------------+--------------------+---------------+--------------+------------+-----------+------+
|c05652fa9d2c4fac9...|9b98b8c7a33c4b65b...|              1|             1|           1|          1|  null|
|9da3f2339b244229a...|4d5c57ea9a6940dd8...|              1|             1|           1|          1|  null|
|fc58860830e648c9b...|9b98b8c7a33c4b65b...|              0|             1|           1|          1|  null|
|f608dc84f5f744a88...|9b98b8c7a33c4b65b...|              1|             1|           1|          1|  null|
|25540a042d7d4aabb...|2298d6c36e964ae4a...|              1|             1|           1|          1|  null|
|f3acf14291c040a5b...|ae264e3637204a6fb...|              1|             1|           1|          1|  null|
|c029ebbd5e584674b...|0b1e1539f2cc45b

In [87]:
tr_event.count()

306534

In [88]:
eventAndoffer_type=tr_event.join(prrtfolio_offer_type,
               tr_event.offer_id == prrtfolio_offer_type.offer_id1, 
               "inner")
eventAndoffer_type=eventAndoffer_type.na.fill(value=0)
eventAndoffer_type=eventAndoffer_type.drop(col("offer_id1"))
eventAndoffer_type.show()

+--------------------+--------------------+---------------+--------------+------------+-----------+------+----+--------+-------------+
|        customer_id1|            offer_id|offer_completed|offer_received|offer_viewed|transaction|amount|bogo|discount|informational|
+--------------------+--------------------+---------------+--------------+------------+-----------+------+----+--------+-------------+
|c05652fa9d2c4fac9...|9b98b8c7a33c4b65b...|              1|             1|           1|          1|   0.0|   1|       0|            0|
|9da3f2339b244229a...|4d5c57ea9a6940dd8...|              1|             1|           1|          1|   0.0|   1|       0|            0|
|fc58860830e648c9b...|9b98b8c7a33c4b65b...|              0|             1|           1|          1|   0.0|   1|       0|            0|
|f608dc84f5f744a88...|9b98b8c7a33c4b65b...|              1|             1|           1|          1|   0.0|   1|       0|            0|
|25540a042d7d4aabb...|2298d6c36e964ae4a...|            

In [89]:
eventAndoffer_type.count()

134002

In [90]:
with_cust_resp=eventAndoffer_type.withColumn('cust_resp', when(((((col('offer_viewed') == 1) &  (col('offer_completed') == 1)) &
                                                       ((col('bogo') == 1) | (col('discount') == 1))) | 
                                                               ((col('offer_received') == 1) & (col('informational') == 1) & (col('amount') > 0) )) ,lit(1)))

with_cust_resp.show()

+--------------------+--------------------+---------------+--------------+------------+-----------+------+----+--------+-------------+---------+
|        customer_id1|            offer_id|offer_completed|offer_received|offer_viewed|transaction|amount|bogo|discount|informational|cust_resp|
+--------------------+--------------------+---------------+--------------+------------+-----------+------+----+--------+-------------+---------+
|c05652fa9d2c4fac9...|9b98b8c7a33c4b65b...|              1|             1|           1|          1|   0.0|   1|       0|            0|        1|
|9da3f2339b244229a...|4d5c57ea9a6940dd8...|              1|             1|           1|          1|   0.0|   1|       0|            0|        1|
|fc58860830e648c9b...|9b98b8c7a33c4b65b...|              0|             1|           1|          1|   0.0|   1|       0|            0|     null|
|f608dc84f5f744a88...|9b98b8c7a33c4b65b...|              1|             1|           1|          1|   0.0|   1|       0|          

In [91]:
with_cust_resp.groupBy('cust_resp').count().show()

+---------+-----+
|cust_resp|count|
+---------+-----+
|     null|50218|
|        1|83784|
+---------+-----+



In [92]:
with_cust_resp=with_cust_resp.select('customer_id1','cust_resp')
with_cust_resp.show()

+--------------------+---------+
|        customer_id1|cust_resp|
+--------------------+---------+
|c05652fa9d2c4fac9...|        1|
|9da3f2339b244229a...|        1|
|fc58860830e648c9b...|     null|
|f608dc84f5f744a88...|        1|
|25540a042d7d4aabb...|        1|
|f3acf14291c040a5b...|        1|
|c029ebbd5e584674b...|     null|
|05ea8d8a95eb4b209...|        1|
|5dca7efd64aa40e9a...|     null|
|559943eeb754463e8...|        1|
|016871ea865d43389...|        1|
|f1a97a4aabba4015b...|        1|
|d7fec020017649a7a...|        1|
|8ca462f1192b4aa29...|     null|
|b3d1075ec03c4c188...|        1|
|f83716ec38f44f038...|        1|
|738943cdecfd45348...|     null|
|dfdd34f61d0645bca...|        1|
|6be0513109fe4d738...|        1|
|ab25fd6fbd5040f88...|        1|
+--------------------+---------+
only showing top 20 rows



In [93]:
with_cust_resp.count()

134002

# Combining Datasets

In [94]:
combined_data1=transcript_1.join(profile_1,transcript_1.customer_id == profile_1.customer_id_gender, "inner")\
.drop('offer_id1','customer_id_gender')
combined_data1=combined_data1.join(portfolio_1,["offer_id"], "inner")\
.drop('offer_id1')
combined_data1=combined_data1.join(with_cust_resp,combined_data1.customer_id == with_cust_resp.customer_id1, "inner")\
.drop('customer_id1','age','drop_col','offer_completed','offer_received','offer_viewed','transaction','became_member_on')
combined_data1=combined_data1.na.fill(value=0)
combined_data1=combined_data1.na.drop()
#combined_data1.printSchema()

In [95]:
combined_data1.count()

979065

In [96]:
combined_data1.groupBy('cust_resp').count().show()

+---------+------+
|cust_resp| count|
+---------+------+
|        1|669434|
|        0|309631|
+---------+------+



In [97]:
transcript_1.count()

306534

In [98]:
profile_1.count()

14825

In [99]:
portfolio_1.count()

10

In [100]:
with_cust_resp.count()

134002

In [101]:
#Dumping the data in mongodb
#ml = spark.read.format("mongo").option("uri","mongodb://localhost:27017/starbucks.Data").load()
#combined_data1.write.format("mongo").option("uri","mongodb://localhost:27017/starbucks.Data").save()

# ===============================================================

In [102]:
cust_resp1=eventAndoffer_type.withColumn('new_col', when((col('bogo') == 1) & (col('offer_received') == 1) & (col('offer_completed') == 1),'comp_bogo')).withColumn('drop_col',lit(1))
cust_resp1=cust_resp1.groupBy('customer_id1').pivot('new_col').sum('drop_col')
cust_resp1=cust_resp1.drop(col('null'))
cust_resp1.show()

+--------------------+---------+
|        customer_id1|comp_bogo|
+--------------------+---------+
|c05652fa9d2c4fac9...|        8|
|9da3f2339b244229a...|        4|
|f608dc84f5f744a88...|        5|
|f3acf14291c040a5b...|        1|
|05ea8d8a95eb4b209...|        4|
|f1a97a4aabba4015b...|        7|
|d7fec020017649a7a...|        3|
|b3d1075ec03c4c188...|        4|
|6be0513109fe4d738...|        4|
|708d8271f758417e9...|        4|
|876ca8a8eacf44cda...|        3|
|72c8dfba6eb44158a...|        5|
|d3c24fa42d0947a4b...|        4|
|5abe3df001c14294a...|        8|
|7d650753a27846f5a...|        4|
|3625554b70d8411bb...|        4|
|fdcb232e66d8452d9...|        9|
|c32b39e1886d4ec2a...|        8|
|e4b82a439eed4edea...|        2|
|36ef980d4deb42ee9...|        5|
+--------------------+---------+
only showing top 20 rows



In [103]:
cust_resp1.groupBy('comp_bogo').count().show()

+---------+-----+
|comp_bogo|count|
+---------+-----+
|        7|  285|
|     null| 5647|
|        6| 1503|
|        9|   50|
|        5|  786|
|        1|  570|
|       10|   60|
|        3| 1101|
|       12|    6|
|        8|  397|
|        2| 3345|
|        4| 3241|
|       11|    3|
+---------+-----+



In [104]:
cust_resp1.count()

16994

In [105]:
cust_resp2=eventAndoffer_type.withColumn('new_col', when((col('discount') == 1) & (col('offer_received') == 1) & (col('offer_completed') == 1),'comp_disc')).withColumn('drop_col',lit(1))
cust_resp2=cust_resp2.groupBy('customer_id1').pivot('new_col').sum('drop_col')
cust_resp2=cust_resp2.drop(col('null'))
cust_resp2.show()

+--------------------+---------+
|        customer_id1|comp_disc|
+--------------------+---------+
|25540a042d7d4aabb...|        2|
|559943eeb754463e8...|        1|
|016871ea865d43389...|        2|
|f83716ec38f44f038...|        4|
|dfdd34f61d0645bca...|        6|
|ab25fd6fbd5040f88...|        6|
|ea1ecddf4814464cb...|        8|
|01e09d713abe4a36a...|        2|
|74450de8dc654737b...|        4|
|37aa48f7035f4b808...|        5|
|6aea2f1dd8ab4ff68...|        4|
|304183fd053441f0a...|        2|
|793a91c8f72d45029...|        6|
|38197aa576534853a...|        3|
|7b68d6af297344258...|        2|
|907e78791e0a4e58b...|        6|
|dd1069bbc7ef423c9...|        5|
|628d90cb330a4c42a...|        8|
|5c73c99208e449129...|        6|
|bea39bf06adf4610b...|       12|
+--------------------+---------+
only showing top 20 rows



In [106]:
two=cust_resp2.join(cust_resp1,['customer_id1'], "inner")
two.show()

+--------------------+---------+---------+
|        customer_id1|comp_disc|comp_bogo|
+--------------------+---------+---------+
|0020ccbbb6d84e358...|        2|        4|
|004b041fbfe448599...|        2|        2|
|0056df74b63b42988...|        3|        2|
|008d7088107b46889...|     null|     null|
|0092a132ead946ceb...|     null|     null|
|00aee28bbb3848dd8...|     null|     null|
|00bbce6533f44ddea...|        4|        2|
|00bc983061d3471e8...|        3|        3|
|00c6035df45840038...|        4|        2|
|00e9f403afa641889...|        5|     null|
|01162252405b4524a...|        5|        3|
|016871ea865d43389...|        2|        1|
|01de69b5edab4b898...|        4|     null|
|01e09d713abe4a36a...|        2|        2|
|01e91c5bc14148b2a...|        8|        2|
|01f33ff554d0443a9...|        2|        1|
|01fe5ec668f241608...|     null|     null|
|0206e1388c34454ca...|     null|        4|
|029e063479234fb1b...|        2|        3|
|02a3aa431c1047be8...|        2|        6|
+----------

In [107]:
two.groupBy('comp_disc').count().show()

+---------+-----+
|comp_disc|count|
+---------+-----+
|        7|  277|
|     null| 5536|
|        6| 1238|
|        9|   49|
|        5|  946|
|        1|  897|
|       10|   43|
|        3| 1641|
|       12|    2|
|        8|  292|
|       11|    3|
|        2| 3141|
|        4| 2929|
+---------+-----+



In [108]:
two.groupBy('comp_bogo').count().show()

+---------+-----+
|comp_bogo|count|
+---------+-----+
|        7|  285|
|     null| 5647|
|        6| 1503|
|        9|   50|
|        5|  786|
|        1|  570|
|       10|   60|
|        3| 1101|
|       12|    6|
|        8|  397|
|        2| 3345|
|        4| 3241|
|       11|    3|
+---------+-----+



In [109]:
cust_resp3=eventAndoffer_type.withColumn('new_col', when((col('informational') == 1) & (col('offer_received') == 1) & (col('amount') > 0),'comp_disc')).withColumn('drop_col',lit(1))
cust_resp3=cust_resp3.groupBy('customer_id1').pivot('new_col').sum('drop_col')
cust_resp3=cust_resp3.withColumnRenamed("null","comp_info")
cust_resp3.show()

+--------------------+---------+
|        customer_id1|comp_info|
+--------------------+---------+
|c05652fa9d2c4fac9...|       10|
|9da3f2339b244229a...|        7|
|fc58860830e648c9b...|        6|
|f608dc84f5f744a88...|        8|
|25540a042d7d4aabb...|        5|
|f3acf14291c040a5b...|        7|
|c029ebbd5e584674b...|        9|
|05ea8d8a95eb4b209...|        5|
|5dca7efd64aa40e9a...|        6|
|559943eeb754463e8...|        3|
|016871ea865d43389...|        8|
|f1a97a4aabba4015b...|        9|
|d7fec020017649a7a...|       10|
|8ca462f1192b4aa29...|        7|
|b3d1075ec03c4c188...|        8|
|f83716ec38f44f038...|       11|
|738943cdecfd45348...|       11|
|dfdd34f61d0645bca...|        8|
|6be0513109fe4d738...|        7|
|ab25fd6fbd5040f88...|        9|
+--------------------+---------+
only showing top 20 rows



In [110]:
cust_resp3.groupBy('comp_info').count().show()

+---------+-----+
|comp_info|count|
+---------+-----+
|        7| 2376|
|        6| 2375|
|        9| 2292|
|        5| 1098|
|        1|    8|
|       10| 2700|
|        3|  209|
|       12|  815|
|        8| 3518|
|       11|  749|
|        2|  106|
|        4|  748|
+---------+-----+



In [111]:
three=two.join(cust_resp3,['customer_id1'], "inner")
three.show()

+--------------------+---------+---------+---------+
|        customer_id1|comp_disc|comp_bogo|comp_info|
+--------------------+---------+---------+---------+
|0020ccbbb6d84e358...|        2|        4|        8|
|004b041fbfe448599...|        2|        2|        5|
|0056df74b63b42988...|        3|        2|        7|
|008d7088107b46889...|     null|     null|       11|
|0092a132ead946ceb...|     null|     null|       12|
|00aee28bbb3848dd8...|     null|     null|        5|
|00bbce6533f44ddea...|        4|        2|        8|
|00bc983061d3471e8...|        3|        3|        6|
|00c6035df45840038...|        4|        2|        8|
|00e9f403afa641889...|        5|     null|        9|
|01162252405b4524a...|        5|        3|       10|
|016871ea865d43389...|        2|        1|        8|
|01de69b5edab4b898...|        4|     null|        7|
|01e09d713abe4a36a...|        2|        2|        5|
|01e91c5bc14148b2a...|        8|        2|       10|
|01f33ff554d0443a9...|        2|        1|    

In [112]:
cust_resp=three.withColumn('new_col', when((col('comp_info')> 0) & (col('comp_bogo') > 0) & (col('comp_disc') > 0),'cust_resp')).withColumn('drop_col',lit(1))
cust_resp=cust_resp.groupBy('customer_id1').pivot('new_col').sum('drop_col')
cust_resp=cust_resp.drop(col('null'))
cust_resp.show()

+--------------------+---------+
|        customer_id1|cust_resp|
+--------------------+---------+
|0020ccbbb6d84e358...|        1|
|004b041fbfe448599...|        1|
|0056df74b63b42988...|        1|
|00bbce6533f44ddea...|        1|
|00bc983061d3471e8...|        1|
|00c6035df45840038...|        1|
|01162252405b4524a...|        1|
|016871ea865d43389...|        1|
|01e09d713abe4a36a...|        1|
|01e91c5bc14148b2a...|        1|
|01f33ff554d0443a9...|        1|
|029e063479234fb1b...|        1|
|02a3aa431c1047be8...|        1|
|02b5aa5725f94bcb9...|        1|
|02e4c9dff62e4857a...|        1|
|02efe413dd194069a...|        1|
|03019e49a5164723a...|        1|
|032aaa79b15e4103a...|        1|
|033fed8bb1504885b...|        1|
|035d5aa3c23a4059a...|        1|
+--------------------+---------+
only showing top 20 rows



In [113]:
cust_resp.groupBy('cust_resp').count().show()

+---------+-----+
|cust_resp|count|
+---------+-----+
|     null| 6963|
|        1|10031|
+---------+-----+



In [114]:
combined_data2=transcript_1.join(profile_1,transcript_1.customer_id == profile_1.customer_id_gender, "inner")\
.drop('offer_id1','customer_id_gender')
combined_data2=combined_data2.join(portfolio_1,["offer_id"], "inner")\
.drop('offer_id1')
combined_data2=combined_data2.join(cust_resp,combined_data1.customer_id == cust_resp.customer_id1, "inner")\
.drop('customer_id1','age','drop_col','offer_completed','offer_received','offer_viewed','transaction','became_member_on')
combined_data2=combined_data2.na.fill(value=0)
combined_data2=combined_data2.na.drop()

In [115]:
combined_data2.count()

116361

In [116]:
combined_data2.groupBy('cust_resp').count().show()

+---------+-----+
|cust_resp|count|
+---------+-----+
|        0|38333|
|        1|78028|
+---------+-----+



In [117]:
combined_data2.printSchema()

root
 |-- offer_id: string (nullable = true)
 |-- event: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- time: long (nullable = true)
 |-- amount: double (nullable = false)
 |-- reward: long (nullable = true)
 |-- F: long (nullable = true)
 |-- M: long (nullable = true)
 |-- O: long (nullable = true)
 |-- 1: long (nullable = true)
 |-- 2: long (nullable = true)
 |-- 3: long (nullable = true)
 |-- 4: long (nullable = true)
 |-- 5: long (nullable = true)
 |-- 6: long (nullable = true)
 |-- 7: long (nullable = true)
 |-- 8: long (nullable = true)
 |-- 9: long (nullable = true)
 |-- 10: long (nullable = true)
 |-- 11: long (nullable = true)
 |-- 12: long (nullable = true)
 |-- income_30: long (nullable = true)
 |-- income_100: long (nullable = true)
 |-- income_110: long (nullable = true)
 |-- income_120: long (nullable = true)
 |-- income_40: long (nullable = true)
 |-- income_50: long (nullable = true)
 |-- income_60: long (nullable = true)
 |-- income_70: long (

In [118]:
#Dumping the data in mongodb
#ml = spark.read.format("mongo").option("uri","mongodb://localhost:27017/starbucks.str_Data").load()
#combined_data2.write.format("mongo").option("uri","mongodb://localhost:27017/starbucks.str_Data").save()

In [119]:
import pandas as pd
import numpy as np
import math
import json
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns